### plotting 有&沒寫考古題者各自的期中考平均分數

In [8]:
# 載入所需函示庫
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as smm
import statsmodels

from matplotlib import pyplot as plt
%matplotlib inline
# 設定圖形大小; DPI越大圖越大
plt.rcParams["figure.dpi"] = 150

In [9]:
# read dataset of doing the old exam or not
df_108_old_exam_before_mid1 = pd.read_excel("PD108_OldExamOrNot.xlsx")
df_109_old_exam_before_mid1 = pd.read_excel("PD109_OldExamOrNot.xlsx")
df_108_old_exam_before_mid2 = pd.read_excel("PD108_OldExam2orNot.xlsx")
df_109_old_exam_before_mid2 = pd.read_excel("PD109_OldExam2orNot.xlsx")

print("head of 108 do the old exam or not before mid1:")
display(df_108_old_exam_before_mid1.head())

head of 108 do the old exam or not before mid1:


,member_id,submission_id
0,14,NaN
1,57,153780.0
2,75,153799.0
3,175,NaN
4,376,NaN


### 分群：有寫考古題、沒寫考古題

In [10]:
# separate those who did and not did the old exams before mid1
df_108_old_exam_true_before_mid1 = df_108_old_exam_before_mid1[df_108_old_exam_before_mid1["submission_id"].isnull() != True]
df_108_old_exam_false_before_mid1 = df_108_old_exam_before_mid1[df_108_old_exam_before_mid1["submission_id"].isnull() == True]
df_109_old_exam_true_before_mid1 = df_109_old_exam_before_mid1[df_109_old_exam_before_mid1["submission_id"].isnull() != True]
df_109_old_exam_false_before_mid1 = df_109_old_exam_before_mid1[df_109_old_exam_before_mid1["submission_id"].isnull() == True]

# before mid2
df_108_old_exam_true_before_mid2 = df_108_old_exam_before_mid2[df_108_old_exam_before_mid2["submission_id"].isnull() != True]
df_108_old_exam_false_before_mid2 = df_108_old_exam_before_mid2[df_108_old_exam_before_mid2["submission_id"].isnull() == True]
df_109_old_exam_true_before_mid2 = df_109_old_exam_before_mid2[df_109_old_exam_before_mid2["submission_id"].isnull() != True]
df_109_old_exam_false_before_mid2 = df_109_old_exam_before_mid2[df_109_old_exam_before_mid2["submission_id"].isnull() == True]

print("108 - those who did the old exams before mid1:")
print(df_108_old_exam_true_before_mid1)

108 - those who did the old exams before mid1:
     member_id  submission_id
1           57       153780.0
2           75       153799.0
10        1025       262423.0
18        1272       154907.0
19        1285       541693.0
23        1376       155785.0
39        1755       246078.0
47        1870       255027.0
48        1871       237954.0
56        1879       255594.0
60        1883       261151.0
62        1887       260075.0
64        1889       261985.0
66        1891       262439.0
70        1895       260070.0
75        1900       246079.0
76        1901       264091.0
99        1956       262224.0
102       1969       262424.0
104       1972       262256.0
108       1980       262206.0
112       1985       262031.0
113       1986       255569.0
115       1989       288108.0
117       1992       262292.0
127       2014       262088.0


### 計算有寫考古題、沒寫考古題兩群各自的期中考平均分數

In [11]:
# read dataset of midterm scores
df_108_mid1 = pd.read_excel("PD108_midterm1_score.xlsx")
df_108_mid2 = pd.read_excel("PD108_midterm2_score.xlsx")
df_109_mid1 = pd.read_excel("PD109_midterm1_score.xlsx")
df_109_mid2 = pd.read_excel("PD109_midterm2_score.xlsx")

print("head of 108 midterm1 score:")
display(df_108_mid1.head())

head of 108 midterm1 score:


,account_id,score
0,1879,100.0
1,1914,100.0
2,1921,100.0
3,1980,100.0
4,1989,100.0


In [12]:
# remove TA's data
TA_108 = [57, 75, 1285, 1758]
TA_109 = [14, 175, 1758, 1767, 587, 590, 707]

df_108_mid1 = df_108_mid1.drop(df_108_mid1[df_108_mid1["account_id"].isin(TA_108)].index)
df_108_mid2 = df_108_mid2.drop(df_108_mid2[df_108_mid2["account_id"].isin(TA_108)].index)
df_109_mid1 = df_109_mid1.drop(df_109_mid1[df_109_mid1["account_id"].isin(TA_109)].index)
df_109_mid2 = df_109_mid2.drop(df_109_mid2[df_109_mid2["account_id"].isin(TA_109)].index)

print("108 mid1 without TAs:")
print(df_108_mid1)

108 mid1 without TAs:
     account_id  score
0          1879  100.0
1          1914  100.0
2          1921  100.0
3          1980  100.0
4          1989  100.0
..          ...    ...
148        1530    NaN
149        1483    NaN
150        1690    NaN
151        2020    NaN
152        1698    NaN

[149 rows x 2 columns]


In [15]:
# separate mid1 scores according to whether the member did the old exams or not
scores_108_old_exam_true_before_mid1 = df_108_mid1[df_108_mid1["account_id"].isin(df_108_old_exam_true_before_mid1["member_id"])]
scores_108_old_exam_false_before_mid1 = df_108_mid1[df_108_mid1["account_id"].isin(df_108_old_exam_false_before_mid1["member_id"])]
scores_109_old_exam_true_before_mid1 = df_109_mid1[df_109_mid1["account_id"].isin(df_109_old_exam_true_before_mid1["member_id"])]
scores_109_old_exam_false_before_mid1 = df_109_mid1[df_109_mid1["account_id"].isin(df_109_old_exam_false_before_mid1["member_id"])]

# mid2
scores_108_old_exam_true_before_mid2 = df_108_mid2[df_108_mid2["account_id"].isin(df_108_old_exam_true_before_mid2["member_id"])]
scores_108_old_exam_false_before_mid2 = df_108_mid2[df_108_mid2["account_id"].isin(df_108_old_exam_false_before_mid2["member_id"])]
scores_109_old_exam_true_before_mid2 = df_109_mid2[df_109_mid2["account_id"].isin(df_109_old_exam_true_before_mid2["member_id"])]
scores_109_old_exam_false_before_mid2 = df_109_mid2[df_109_mid2["account_id"].isin(df_109_old_exam_false_before_mid2["member_id"])]

print("108 - mid1 scores of those who did the old exams before mid1:")
print(scores_108_old_exam_true_before_mid1)

108 - mid1 scores of those who did the old exams before mid1:
     account_id  score
0          1879  100.0
3          1980  100.0
4          1989  100.0
15         1871   98.0
22         1889   96.0
23         1755   94.0
27         1870   94.0
47         2014   82.0
51         1901   78.0
52         1891   78.0
56         1895   76.0
57         1986   74.0
64         1956   70.0
65         1969   70.0
67         1025   68.0
81         1887   60.0
83         1883   58.0
92         1972   46.0
99         1900   40.0
100        1272   38.0
110        1985   34.0
112        1376   32.0
119        1992   22.0


In [23]:
# compute average mid1 score of each group
avg_108_old_exam_true_before_mid1 = scores_108_old_exam_true_before_mid1["score"].mean()
avg_108_old_exam_false_before_mid1 = scores_108_old_exam_false_before_mid1["score"].mean()
avg_109_old_exam_true_before_mid1 = scores_109_old_exam_true_before_mid1["score"].mean()
avg_109_old_exam_false_before_mid1 = scores_109_old_exam_false_before_mid1["score"].mean()

# mid2
avg_108_old_exam_true_before_mid2 = scores_108_old_exam_true_before_mid2["score"].mean()
avg_108_old_exam_false_before_mid2 = scores_108_old_exam_false_before_mid2["score"].mean()
avg_109_old_exam_true_before_mid2 = scores_109_old_exam_true_before_mid2["score"].mean()
avg_109_old_exam_false_before_mid2 = scores_109_old_exam_false_before_mid2["score"].mean()


print("108 - average mid1 score of those who did the old exams before mid1:", avg_108_old_exam_true_before_mid1)
print("108 - average mid1 score of those who did not do the old exams before mid1:", avg_108_old_exam_false_before_mid1)
print("109 - average mid1 score of those who did the old exams before mid1:", avg_109_old_exam_true_before_mid1)
print("109 - average mid1 score of those who did not do the old exams before mid1:", avg_109_old_exam_false_before_mid1)
print("\n")

print("108 - average mid2 score of those who did the old exams before mid2:", avg_108_old_exam_true_before_mid2)
print("108 - average mid2 score of those who did not do the old exams before mid2:", avg_108_old_exam_false_before_mid2)
print("109 - average mid2 score of those who did the old exams before mid2:", avg_109_old_exam_true_before_mid2)
print("109 - average mid2 score of those who did not do the old exams before mid2:", avg_109_old_exam_false_before_mid2)

108 - average mid1 score of those who did the old exams before mid1: 69.91304347826087
108 - average mid1 score of those who did not do the old exams before mid1: 62.94230769230769
109 - average mid1 score of those who did the old exams before mid1: 86.14814814814815
109 - average mid1 score of those who did not do the old exams before mid1: 70.6140350877193


108 - average mid2 score of those who did the old exams before mid2: 62.642857142857146
108 - average mid2 score of those who did not do the old exams before mid2: 59.45454545454545
109 - average mid2 score of those who did the old exams before mid2: 53.391304347826086
109 - average mid2 score of those who did not do the old exams before mid2: 55.48076923076923
